# Cluster errors to identify the type of errors that can appear in solver reports 

# Table of Contents

1. [Introduction](#Introduction)
2. [Import Packages](#Import_packages)
3. [Load the clean solver data saved by 'PreprocessSolverErrorData' notebook](#load_clean_data)
4. [Filter data using Solver / datetime](#filter)
5. [Word to Vector Conversion using Continuous Bag of Words model (CBOW)](#word2vec)
6. [Sentence (error message) to vector conversion](#sent2vec)
7. [Clustering using DBScan](#clustering)
8. [Get cluster statistics such as : "pattern", "mean_length", "mean_similarity"](#cluster_stats)
9. [Save clustered data to Ceph](#save_to_ceph)
10. [View data from each cluster](#view_data)
 1. [Cluster No. 0: FileNotFoundError](#c0)
 2. [Cluster No. 1: UnableToExecuteGccError](#c1)
 3. [Cluster No. 3: NoMatchingDistributionFoundError](#c3)
11. [Clusters with more than one error](#clusters_with_more_than_one_error)
 1. [Cluster No. 10: ImportError, HTTPError](#c10)
 2. [Cluster No. 106: CalledProcessError, FileNotFoundError, KeyError, RuntimeError](#c106)
 3. [Cluster No. 116:  ConnectionError, OSError, MaxRetryError, DistutilsError, ResponseError](#c16)
 4. [Cluster No. 7: CheckTheLogsError : Need further exploring](#c7) 

## Introduction  <a id='Introduction'></a>

The purpose of this notebook is to cluster solver errors so that we can derive context on why dependencies cannot be solved in order to better advise users on why something cannot be used.

#### Summary :
- Preprocessed data by [PreprocessSolverErrorData](./PreprocessSolverErrorData.ipynb) notebook is loaded.
- Each word in converted into a vector using [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) (Continuous Bag of Words model). 
- Each error message is then converted into a vector(Sentence2vec using word2vec model).
- Clustering is done using [DBScan](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).
- Cluster statistics such as "pattern", "mean_length" and "mean_similarity" is calculated.
- Error Class is defined and added to the dataframe.
- Saved the classified error data to Ceph.

## Import packages <a id='Import_packages'></a>

In [1]:
import pandas as pd
import multiprocessing
import pickle
import numpy as np
import difflib
import regex as re
import boto3
import os

from math import sqrt
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from gensim.models import Word2Vec
from kneed import KneeLocator
from string import punctuation    

In [2]:
pd.set_option('max_colwidth', 4000)
pd.set_option('display.max_rows', 200)

In [3]:
cpu_number = multiprocessing.cpu_count()
w2v_window= 7

## Load the clean solver data saved by 'PreprocessSolverErrorData' notebook <a id='load_clean_data'></a>

In [4]:
preprocessed_filename = 'error-clean-data.csv'

In [5]:
entire_error_df = pd.read_csv(preprocessed_filename)

In [6]:
entire_error_df.head()

index                     document_id  \
0      0  solver-fedora-31-py37-0022caa4   
1      1  solver-fedora-31-py37-003d9de4   
2      2  solver-fedora-31-py37-004344f0   
3      3  solver-fedora-31-py37-005ab6b4   
4      4  solver-fedora-31-py37-0063867d   

                                                                                                                                                                      command  \
0      /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps Cython===0.23.4 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
1     /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps word2vec===0.7.1 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
2   /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps papermill===0.13.3 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
3  /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps pomegranate===0.7.7 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
4       /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps catboost===0.6 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   

  package_name package_version                 solver  \
0       Cython          0.23.4  solver-fedora-31-py37   
1     word2vec           0.7.1  solver-fedora-31-py37   
2    papermill          0.13.3  solver-fedora-31-py37   
3  pomegranate           0.7.7  solver-fedora-31-py37   
4     catboost             0.6  solver-fedora-31-py37   

                     datetime  \
0  2020-08-05T21:54:08.679464   
1  2020-08-18T17:33:54.656033   
2  2020-08-11T04:00:49.015446   
3  2020-08-01T02:36:45.424428   
4  2020-08-01T02:36:25.821970   

                                                                                                                                                                                                                                                                                                                                                                                        environment  \
0  {'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}   
1  {'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}   
2  {'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}   
3  {'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}   
4  {'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_

In [7]:
len(entire_error_df)

19219

## Filter data using Solver / datetime <a id='filter'></a>

In [8]:
def filter_data(entire_error_df, solver_name=None, start_date='2019-12-27',end_date='2020-01-14', mode='solver'):
    if mode == 'solver':
        error_df = entire_error_df.loc[entire_error_df['solver'] == solver_name]
    elif mode == 'datetime':
        mask = (entire_error_df['datetime'] >= start_date) & (entire_error_df['datetime'] <= end_date)
        error_df = entire_error_df.loc[mask]
    elif mode == 'all':
        error_df = entire_error_df
    return error_df

In [9]:
entire_error_df['solver'].unique()

array(['solver-fedora-31-py37', 'solver-fedora-31-py38',
       'solver-fedora-32-py37', 'solver-fedora-32-py38',
       'solver-rhel-8-py36'], dtype=object)

In [10]:
#error_df = filter_data(entire_error_df, solver_name = 'solver-fedora-31-py37', mode='solver')
#error_df = filter_data(entire_error_df, start_date='2019-12-24',end_date='2020-01-14', mode='datetime')
error_df = filter_data(entire_error_df, mode = 'all')

In [11]:
len(error_df)

19219

### Extract tokenized_clustering_data for clustering

In [12]:
clean_clustering_data = error_df['tokenized_clustering_data']

## Word to Vector Conversion using Continuous Bag of Words model (CBOW) <a id='word2vec'></a>

In [13]:
print('Number of rows in training data :', len(clean_clustering_data))

Number of rows in training data : 19219


In [14]:
def detect_embedding_size(tokens):
    flat_list = [item for row in tokens for item in row]
    vocab = set(flat_list)
    embedding_size = round(len(vocab) ** (2/3))
    if embedding_size >= 400:
        embedding_size = 400
    return embedding_size

w2v_size = detect_embedding_size(clean_clustering_data)

In [15]:
def tokens_vectorization(clustering_data, w2v_size, w2v_window, cpu_number, model_name):
    iterations = 100
    word2vec = Word2Vec(clustering_data,
                           size = w2v_size, 
                           window = w2v_window, 
                           min_count=1, 
                           workers = cpu_number,
                           iter=iterations)
    word2vec.save(model_name)
    return word2vec

In [16]:
word2vec = tokens_vectorization(clean_clustering_data, 
                                 w2v_size = w2v_size, 
                                 w2v_window= w2v_window, 
                                 cpu_number = cpu_number, 
                                 model_name='../models/word2vec.model')

2020-09-11 11:00:22,037 [85526] WARNING  gensim.models.base_any2vec:720: [JupyterRequire] consider setting layer size to a multiple of 4 for greater performance
2020-09-11 11:00:22,043 [85526] WARNING  gensim.models.word2vec:1378: [JupyterRequire] Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.


## Sentence (error message) to vector conversion <a id='sent2vec'></a>

sum all content words in the documents and divide by the number of content words.

In [17]:
def sentence_vectorization(clustering_data, word2vec):
    sent2vec = []
    for sent in clustering_data:
        sent_vec = []
        numw = 0
        for w in sent:
            try:
                sent_vec = word2vec[w] if numw == 0 else np.add(sent_vec, word2vec[w])
                numw += 1
            except Exception:
                pass
        sent2vec.append(np.asarray(sent_vec) / numw)   
    return np.vstack(sent2vec)

In [18]:
sent2vec = sentence_vectorization(clean_clustering_data, word2vec)

2020-09-11 11:00:31,342 [85526] WARNING  py.warnings:99: [JupyterRequire] /home/fmurdaca/.local/share/virtualenvs/solver-errors-reporter-w-y7MFT4/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  



## Clustering using DBScan  <a id='clustering'></a>

Based on a set of points DBSCAN groups together points that are close to each other based on a distance measurement(epsilon) and a minimum number of points. It also marks as outliers the points that are in low-density regions.

Find the avg_distances using NearestNeighbors between the data points.

In [19]:
def kneighbors(sent2vec):
    k = round(sqrt(len(sent2vec)))
    neigh = NearestNeighbors(n_neighbors=k)
    nbrs = neigh.fit(sent2vec)
    distances, indices = nbrs.kneighbors(sent2vec)
    distances = [np.mean(d) for d in np.sort(distances, axis=0)]
    return distances

avg_distances = kneighbors(sent2vec)

Calculate epsilon, which is the linkage distance threshold above which, clusters will not be merged.

In [20]:
def epsilon_search(distances):
    kneedle = KneeLocator(distances, list(range(len(distances))))
    epsilon = max(kneedle.all_elbows) if (len(kneedle.all_elbows) > 0) else 1
    return epsilon

In [21]:
epsilon = epsilon_search(avg_distances)

DBScan Clustering using epsilon and min_samples as 1

In [22]:
def dbscan(epsilon, min_samples, cpu_number, sent2vec):
    cluster_labels = DBSCAN(eps=epsilon,
                            min_samples= min_samples,
                            n_jobs=cpu_number).fit_predict(sent2vec)
    return cluster_labels

In [23]:
#cluster_labels = hierarchical(epsilon, sent2vec)
cluster_labels = dbscan(epsilon, 1, cpu_number, sent2vec)

In [24]:
len(cluster_labels)

19219

In [25]:
error_df['cluster_no.'] = cluster_labels

## Get cluster statistics such as : "pattern", "mean_length", "mean_similarity" <a id='cluster_stats'></a>

In [26]:
def clustered_output(error_df, mode='INDEX'):
    groups, unique_rows = {}, {}
    for key, value in error_df.groupby(['cluster_no.']):
        unique_rows[str(key)] = set(value['clustering_data'])
        if mode == 'ALL':
            groups[str(key)] = value.to_dict(orient='records')
        elif mode == 'Tokenized':
            groups[str(key)] = value['tokenized_clustering_data'].values.tolist()
        elif mode == 'CLEANED':
            groups[str(key)] = value['clustering_data'].values.tolist()
    return groups, unique_rows

In [27]:
table = str.maketrans(punctuation, ' '*len(punctuation))

def find_matching_blocks(strings):
    curr = strings[0]
    curr = curr.replace('ERROR', '')
    curr = curr.replace('Command exited with non-zero status code (1):', '')
    if len(strings) == 1:
        #return curr
        return curr.translate(table).strip()
    else:
        cnt = 1
        for i in range(cnt, len(strings)):
            matches = difflib.SequenceMatcher(None, curr, strings[i])
            common = []
            for match in matches.get_matching_blocks():
                common.append(curr[match.a:match.a + match.size])
            curr = ''.join(common)
            cnt = cnt + 1
            if cnt == len(strings):
                break
        if curr == '':
            'NO COMMON PATTERNS HAVE BEEN FOUND'
        #return curr
        return curr.translate(table).strip()

def get_similarity(rows):
    s = []
    for i in range(0, len(rows)):
        s.append(difflib.SequenceMatcher(None, rows[0], rows[i]).ratio() * 100)
    return s

In [28]:
STATISTICS = ["cluster_name", "cluster_size", "pattern", 'CLASS', "mean_similarity"]

def statistics(error_df, output_mode='frame'):
    """
    Returns dictionary with statistic for all clusters
    "cluster_name" - name of a cluster
    "cluster_size" = number of log messages in cluster
    "pattern" - all common substrings in messages in the cluster
    "mean_length" - average length of log messages in cluster
    "mean_similarity" - average similarity of log messages in cluster
    (calculated as the levenshtein distances between the 1st and all other log messages)
    :param clustered_df:
    :param output_mode: frame | dict
    :return:
    """
    clusters = []
    clustered_df, unique_rows = clustered_output(error_df, mode='CLEANED')
    clustered_df_class, unique_rows = clustered_output(error_df, mode='Tokenized')
    for item in clustered_df:
        row = clustered_df[item]
        matcher = find_matching_blocks(row)
        class_matcher = find_matching_blocks(clustered_df_class[item])
        similarity = get_similarity(row)
        clusters.append([item,
                         len(row),
                         matcher,
                         class_matcher,
                         #unique_rows[item],
                         #np.mean(lengths),
                         np.mean(similarity)])
    df = pd.DataFrame(clusters, columns=STATISTICS).round(2).sort_values(by='cluster_size', ascending=False)
    if output_mode == 'frame':
        return df
    else:
        return df.to_dict(orient='records')

In [29]:
stat = statistics(error_df, output_mode='frame')
stat_df = pd.DataFrame.from_dict(stat)

In [30]:
print('Number of clusters : ', len(stat_df))

Number of clusters :  40


Generate CLASS label

In [31]:
def get_class_label(stat_df):
    class_labels = []
    number_of_errors = []
    MachineDefinedError = []
    for item in stat_df['CLASS']:
        if "Error" in item.split():
            item = item.replace('Error', '')
        row = item.split()
        #if len(row) > 1 and len(re.findall(r'Error', str(row))) < 2:
        if not re.search('(\w\w*Error)', item):
            MachineDefinedError.append('NO')
            item = ''
            for word in row:
                item += word[0].upper() + word[1:]
            item += "Error"
        else:
            if len(re.findall(r'Error', str(row))) > 1:
                item = ', '.join(row)
            else:
                item = ''.join(row)
            MachineDefinedError.append('YES')
        class_labels.append(item)
        number_of_errors.append(len(re.findall(r'Error', str(item))))
    return class_labels, number_of_errors, MachineDefinedError

In [32]:
class_labels, number_of_errors, MachineDefinedError = get_class_label(stat_df)

In [33]:
stat_df['number_of_errors'] = number_of_errors
stat_df['MachineDefinedError?'] = MachineDefinedError
stat_df['CLASS'] = class_labels

In [34]:
stat_df.sort_values(by='cluster_size', ascending=False)

,cluster_name,cluster_size,pattern,CLASS,mean_similarity,number_of_errors,MachineDefinedError?
6,6,9718,ModuleNotFoundError,ModuleNotFoundError,100.00,1,YES
3,3,3214,No matching distribution found for,NoMatchingDistributionFoundError,84.82,1,NO
0,0,1345,SyntaxError,SyntaxError,100.00,1,YES
2,2,1294,unable to execute gcc No such file or directory,UnableToExecuteGccError,100.00,1,NO
5,5,1229,AttributeError,AttributeError,100.00,1,YES
1,1,432,FileNotFoundError,FileNotFoundError,100.00,1,YES
7,7,387,Command errored out with exit status 1,CheckTheLogsError,99.90,1,NO
9,9,281,CUDA could not be found on your system,CUDACouldNotBeFoundError,100.00,1,NO
12,12,237,RuntimeError,RuntimeError,100.00,1,YES
14,14,205,Failed building wheel for,FailedBuildingWheelError,89.27,1,NO


In [35]:
error_df['CLASS'] = error_df['cluster_no.'].map(stat_df['CLASS'])
error_df['number_of_errors'] = error_df['cluster_no.'].map(stat_df['number_of_errors'])
error_df['MachineDefinedError?'] = error_df['cluster_no.'].map(stat_df['MachineDefinedError?'])

## Save clustered data to Ceph <a id='save_to_ceph'></a>

In [36]:
import os
THOTH_S3_ENDPOINT_URL = os.environ['THOTH_S3_ENDPOINT_URL']
THOTH_CEPH_KEY_ID = os.environ['THOTH_CEPH_KEY_ID']
THOTH_CEPH_SECRET_KEY = os.environ['THOTH_CEPH_SECRET_KEY']

In [37]:
error_df.head(50)

index                     document_id  \
0       0  solver-fedora-31-py37-0022caa4   
1       1  solver-fedora-31-py37-003d9de4   
2       2  solver-fedora-31-py37-004344f0   
3       3  solver-fedora-31-py37-005ab6b4   
4       4  solver-fedora-31-py37-0063867d   
5       5  solver-fedora-31-py37-007b905b   
6       6  solver-fedora-31-py37-008441f2   
7       7  solver-fedora-31-py37-0084da4c   
8       8  solver-fedora-31-py37-00912d02   
9       9  solver-fedora-31-py37-0095681b   
10     10  solver-fedora-31-py37-00a42eae   
11     11  solver-fedora-31-py37-00a9760f   
12     12  solver-fedora-31-py37-00c0376f   
13     13  solver-fedora-31-py37-0112bd2f   
14     14  solver-fedora-31-py37-01698085   
15     15  solver-fedora-31-py37-01786d78   
16     16  solver-fedora-31-py37-018dd626   
17     17  solver-fedora-31-py37-018eac13   
18     18  solver-fedora-31-py37-01b7f08c   
19     19  solver-fedora-31-py37-01ecc797   
20     20  solver-fedora-31-py37-01efd8ae   
21     21  solver-fedora-31-py37-0202ab4f   
22     22  solver-fedora-31-py37-026a58a1   
23     23  solver-fedora-31-py37-026c893b   
24     24  solver-fedora-31-py37-027ae674   
25     25  solver-fedora-31-py37-028a13ed   
26     26  solver-fedora-31-py37-02b67279   
27     27  solver-fedora-31-py37-02c9ad7d   
28     28  solver-fedora-31-py37-02d0df73   
29     29  solver-fedora-31-py37-02e97336   
30     30  solver-fedora-31-py37-02f2b727   
31     31  solver-fedora-31-py37-02f81a74   
32     32  solver-fedora-31-py37-0306e4d0   
33     33  solver-fedora-31-py37-0325bf92   
34     34  solver-fedora-31-py37-0329afc9   
35     35  solver-fedora-31-py37-032d7e36   
36     36  solver-fedora-31-py37-0363bd23   
37     37  solver-fedora-31-py37-03735f37   
38     38  solver-fedora-31-py37-037fb20e   
39     39  solver-fedora-31-py37-03dc584b   
40     40  solver-fedora-31-py37-03f6090c   
41     41  solver-fedora-31-py37-042b6a9b   
42     42  solver-fedora-31-py37-043ff21b   
43     43  solver-fedora-31-py37-044d4c05   
44     44  solver-fedora-31-py37-045af1b5   
45     45  solver-fedora-31-py37-0487505a   
46     46  solver-fedora-31-py37-04bf4ba9   
47     47  solver-fedora-31-py37-04d42286   
48     48  solver-fedora-31-py37-04d65110   
49     49  solver-fedora-31-py37-054edf0b   

                                                                                                                                                                                                                                                      command  \
0                                                                                      /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps Cython===0.23.4 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
1                                                                                     /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps word2vec===0.7.1 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
2                                                                                   /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps papermill===0.13.3 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
3                                                                                  /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps pomegranate===0.7.7 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
4                                                                                       /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps catboost===0.6 --index-url "https://pypi.org/simple"  --trusted-host pypi.org   
5                                                                                         /home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-di

In [96]:
from io import StringIO
import datetime

DATETIME = datetime.datetime.utcnow()
DATE = DATETIME.strftime('%Y-%m-%d')

def store_csv_to_ceph(error_df):
    csv_buffer = StringIO()
    error_df = error_df.drop(columns =['index', "command", 'environment', 'message','split_message', 'Error_info', 'command_info', 
                                       'cwd', 'Complete_output','ERROR', 'Exception', 'specific_error', 'tokenized_clustering_data',
                                      "context_message"])
    error_df.to_csv(csv_buffer, header=False, index=False)
    bucket = 'thoth'
    s3_resource = boto3.resource('s3',
                        endpoint_url= THOTH_S3_ENDPOINT_URL,
                        aws_access_key_id = THOTH_CEPH_KEY_ID,
                        aws_secret_access_key= THOTH_CEPH_SECRET_KEY)
    s3_resource.Object(bucket, f'data/ocp-stage/solver-error-context/solver-error-context-{DATE}.csv').put(Body=csv_buffer.getvalue())
    
    
    bucket = 'DH-PLAYPEN'
    s3_resource = boto3.resource('s3',
                        endpoint_url= THOTH_S3_ENDPOINT_URL,
                        aws_access_key_id = THOTH_CEPH_KEY_ID,
                        aws_secret_access_key= THOTH_CEPH_SECRET_KEY)
    s3_resource.Object(bucket, f'data/ocp-stage/solver-error-context/solver-error-context-{DATE}.csv').put(Body=csv_buffer.getvalue())

In [97]:
store_csv_to_ceph(error_df)

## View data from each cluster <a id='view_data'></a>

In [38]:
def get_data_from_cluster(df_processed, clusters, cluster_number):
    """Get data from a specific cluster."""
    indices = [i for i, x in enumerate(clusters) if x == cluster_number]
    df_grouped = df_processed.iloc[indices]
    print(len(df_grouped))
    return df_grouped

def split_log(log_messages):
    """Split log."""
    log_messages = log_messages.split('\n')
    return log_messages

### Check packages with error for a specific cluster	<a id='c1'></a>

In [52]:
cluster_ = 0

get_data_from_cluster(error_df, cluster_labels, cluster_)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']]

1345


package_name package_version                 solver  \
0           Cython          0.23.4  solver-fedora-31-py37   
1         word2vec           0.7.1  solver-fedora-31-py37   
17      sqlalchemy           0.3.4  solver-fedora-31-py37   
20         roundup           1.6.0  solver-fedora-31-py37   
26           Flask           0.7.1  solver-fedora-31-py37   
...            ...             ...                    ...   
19183        numpy           1.3.0     solver-rhel-8-py36   
19201   sqlalchemy           0.2.8     solver-rhel-8-py36   
19204     networkx            0.34     solver-rhel-8-py36   
19216   sqlalchemy           0.3.2     solver-rhel-8-py36   
19217      xgboost          0.4a23     solver-rhel-8-py36   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Check errors for a specific package <a id='c0'></a>

In [51]:
package_name = "fbprohet"

for cluster_number in stat_df['cluster_name'].values:
    print(cluster_number)
    cluster_df = get_data_from_cluster(error_df, cluster_labels, cluster_number)[['package_name', 'package_version', 'solver','message', 
                                                        'specific_error', 'CLASS', 'MachineDefinedError?']]
    
    records = cluster_df[cluster_df['package_name'] == "fbprohet"].shape[0]

    
    if records > 0:
        
        print("records", records)

6
0
3
0
0
0
2
0
5
0
1
0
7
0
9
0
12
0
14
0
4
0
11
0
13
0
21
0
8
0
19
0
20
0
16
0
10
0
24
0
18
0
17
0
38
0
31
0
34
0
32
0
30
0
29
0
25
0
27
0
26
0
23
0
15
0
36
0
28
0
33
0
22
0
37
0
35
0
39
0


### Checkin errors type for a specific cluster <a id='c0'></a>

In [109]:
number_cluster = 0

for un in get_data_from_cluster(error_df, cluster_labels, number_cluster)[['package_name', 'package_version', 'solver','message', 
                                                    'specific_error', 'CLASS', 'MachineDefinedError?']].sort_values(by='specific_error')['specific_error'].unique():
    
    print(un)
    print("Count: ", error_df[error_df['specific_error'] == un].shape[0])

1345
["SyntaxError: Missing parentheses in call to 'exec'"]
Count:  52
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Created:', bat_path)?"]
Count:  10
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Unsupported operating system:',os.name)?"]
Count:  5
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Windows users please use github installation.')?"]
Count:  47
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('adding pytz')?"]
Count:  5
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('found system libevent for', sys.platform)?"]
Count:  6
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('installing data to', datapath)?"]
Count:  12
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print('send: GET CACHE %s'%uri)?"]
Count:  5
["SyntaxError: Missing parentheses in call to 'print'. Did you mean print(80 * '*')?"]
Co